In [ ]:
import os
from pathlib import Path
from source.naver_geocode import NAVERGeocoding
import pandas as pd
from tqdm import tqdm
save_path = "./anmawon/local_update.csv"

In [ ]:
if Path(save_path).exists():
    data = pd.read_csv("./test.csv")
else:
    region1 = pd.read_csv("./anmawon/regions/Busan.csv")
    region1 = region1.assign(region="부산")

    region2 = pd.read_csv("./anmawon/regions/Daegu.csv")
    region2 = region2.assign(region="대구")

    region3 = pd.read_csv("./anmawon/regions/GyeongsangNorth.csv")
    region3 = region3.assign(region="경북")

    region4 = pd.read_csv("./anmawon/regions/GyeongsangSouth.csv")
    region4 = region4.assign(region="경남")

    region5 = pd.read_csv("./anmawon/regions/Incheon.csv")
    region5 = region5.assign(region="인천")

    region6 = pd.read_csv("./anmawon/regions/Ulsan.csv")
    region6 = region6.assign(region="울산")

    total = pd.concat([region1,region2,region3,region4,region5,region6], ignore_index=True)
    total = total[["name", "addr", "phone", "link", "region"]]

    #centeral = pd.read_csv("./anmawon/regions/center.csv")

    prepared = pd.read_csv("./anmawon/manual_edit_anmawon_geolocated.csv")
    prepared.rename(columns={"지역":"region","사업장명":"name","주소":"addr","전화번호":"phone","링크":"link"}, inplace=True)
    not_regioned = ["서울", "경기", "광주", "대전", "강원", "충북", "충남", "대전", "전북", "전남", "제주"]
    prepared = prepared.loc[prepared['region'].isin(not_regioned)]
    data = pd.concat([total, prepared], ignore_index=True)


In [ ]:
geocoding = NAVERGeocoding(secret_key_path="../ts_code/sources/json/secret.json", geocode_url="https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode")

In [ ]:
data.fillna("", inplace=True)
for idx, row in tqdm(data.iterrows()):
    if row["x"] and row["y"] and row["NAVER_addr"] :
        continue  # fill empty cases only
    try:
        addr = row["addr"]
        x, y, naver_addr = geocoding(addr)
        data.loc[idx, ["x", "y", "NAVER_addr"]] = x, y, naver_addr
    except Exception as e:  # 예외 발생한 아웃라이어에 대해서는 직접 수정 필요
        print(idx, e)
        pass
data.to_csv("./test.csv",sep=",", header=True, index=False)